In [68]:
import numpy as np
import pandas as pd

In [69]:
EU_COUNTRIES = eu_countries = [
    'Austria',
    'Belgium',
    'Bulgaria',
    'Croatia',
    'Cyprus',
    'Czechia',
    'Denmark',
    'Estonia',
    'Finland',
    'France',
    'Germany',
    'Greece',
    'Hungary',
    'Ireland',
    'Italy',
    'Latvia',
    'Lithuania',
    'Luxembourg',
    'Netherlands',
    'Poland',
    'Portugal',
    'Romania',
    'Slovakia',
    'Slovenia',
    'Spain',
    'Sweden'
]

comtrade = pd.read_csv('data/comtrade_2019.csv', index_col=0)
exporters = comtrade[['ISO', 'Country']].drop_duplicates().set_index('ISO')['Country'].to_dict()

comtrade['EU'] = comtrade['ImportCountry'].apply(lambda x: 'EU' if x in EU_COUNTRIES else 'Other')
print("EU Imports:", comtrade[comtrade['EU'] == 'EU']['Weight'].sum() / 60)
comtrade_data = comtrade.pivot_table(index='Country', columns='EU', values='Weight', aggfunc='sum').T.to_dict()

EU Imports: 48285526.21666667


In [70]:
geo = pd.read_csv('data/jebena_geo_map_dataset.csv', thousands=',')
geo.columns = ['ssu_code', 'label', 'value', 'variable']
geo['country_code'] = geo['ssu_code'].apply(lambda x: x[:2])
geo.tail()

,ssu_code,label,value,variable,country_code
167587,zm_ssu_eastern_zambia,Eastern Zambia,1213580,estimated_arabica_production_in_kg,zm
167588,zm_ssu_western_zambia,Western Zambia,0,estimated_arabica_farmer_population,zm
167589,zm_ssu_western_zambia,Western Zambia,0,estimated_arabica_production_in_kg,zm
167590,zm_ssu_western_zambia,Western Zambia,0,estimated_robusta_farmer_population,zm
167591,zm_ssu_western_zambia,Western Zambia,0,estimated_robusta_production_in_kg,zm


In [71]:
country_code_mapping = {
    'au': 'AUS',  # Australia
    'bi': 'BDI',  # Burundi
    'bo': 'BOL',  # Bolivia
    'br': 'BRA',  # Brazil
    'cd': 'COD',  # Democratic Republic of the Congo
    'ci': 'CIV',  # Côte d'Ivoire
    'cn': 'CHN',  # China
    'co': 'COL',  # Colombia
    'cr': 'CRI',  # Costa Rica
    'ec': 'ECU',  # Ecuador
    'et': 'ETH',  # Ethiopia
    'gt': 'GTM',  # Guatemala
    'hn': 'HND',  # Honduras
    'id': 'IDN',  # Indonesia
    'in': 'IND',  # India
    'ke': 'KEN',  # Kenya
    'la': 'LAO',  # Laos
    'mx': 'MEX',  # Mexico
    'my': 'MYS',  # Malaysia
    'ni': 'NIC',  # Nicaragua
    'pa': 'PAN',  # Panama
    'pe': 'PER',  # Peru
    'pg': 'PNG',  # Papua New Guinea
    'rw': 'RWA',  # Rwanda
    'sv': 'SLV',  # El Salvador
    'th': 'THA',  # Thailand
    'tl': 'TLS',  # Timor-Leste
    'tz': 'TZA',  # Tanzania
    'ug': 'UGA',  # Uganda
    'vn': 'VNM',  # Vietnam
    'ye': 'YEM',  # Yemen
    'zm': 'ZMB',  # Zambia
}
geo['country'] = geo['country_code'].apply(lambda x: exporters.get(country_code_mapping.get(x), 'Other'))
geo['country'].unique()

array(['Other', 'Burundi', 'Bolivia', 'Brazil', 'Congo (Kinshasa)',
       "Cote d'Ivoire", 'China', 'Colombia', 'Costa Rica', 'Ecuador',
       'Ethiopia', 'Guatemala', 'Honduras', 'Indonesia', 'India', 'Kenya',
       'Laos', 'Mexico', 'Nicaragua', 'Panama', 'Peru',
       'Papua New Guinea', 'Rwanda', 'El Salvador', 'Tanzania', 'Uganda',
       'Vietnam', 'Yemen'], dtype=object)

In [72]:
countries = geo.pivot_table(
    index='country',
    columns='variable',
    values='value',
    aggfunc='sum'
)
countries['num_farmers'] = countries['estimated_arabica_farmer_population'] + countries['estimated_robusta_farmer_population']
countries['est_production'] = countries['estimated_arabica_production_in_kg'] + countries['estimated_robusta_production_in_kg']
production_data = countries[['num_farmers', 'est_production']].T.to_dict()

In [97]:
class Farmer:
    def __init__(self, production):
        self.production = production
        self.sold_to_middlemen = 0

class Middleman:
    def __init__(self, capacity):
        self.capacity = capacity
        self.supplied_to_exporters = 0

class Exporter:
    def __init__(self, capacity):
        self.capacity = capacity
        self.supplied_to_eu = 0 

In [98]:
def generate_farmers(num_farmers, total_production):
    production_distribution = np.random.lognormal(
        mean=np.log(total_production / num_farmers), sigma=0.5, size=num_farmers
    )
    production_distribution = np.maximum(production_distribution, 10)  # Minimum 10 kg
    production_distribution = production_distribution / production_distribution.sum() * total_production
    return [Farmer(production) for production in production_distribution]


In [100]:
def generate_middlemen(num_middlemen, total_production):
    capacity_distribution = np.random.normal(
        loc=0.01 * total_production, scale=0.005 * total_production, size=num_middlemen
    )
    capacity_distribution = np.clip(capacity_distribution, 1000, None)
    return [Middleman(capacity) for capacity in capacity_distribution]

def generate_exporters(num_exporters, total_exports):
    pareto_alpha = 1.5
    capacity_distribution = (np.random.pareto(pareto_alpha, num_exporters) + 1) * max(20000, 0.001 * total_exports)
    capacity_distribution = np.clip(capacity_distribution, 20000, None)
    return [Exporter(capacity) for capacity in capacity_distribution]

# Interaction functions
def assign_farmers_to_middlemen(farmers, middlemen):
    for farmer in farmers:
        while farmer.production > 0:
            middleman = np.random.choice(middlemen)
            transfer = min(farmer.production, middleman.capacity - middleman.supplied_to_exporters, 10)
            middleman.supplied_to_exporters += transfer
            farmer.production -= transfer

def assign_middlemen_to_exporters(middlemen, exporters):
    for middleman in middlemen:
        while middleman.supplied_to_exporters > 0:
            exporter = np.random.choice(exporters)
            transfer = min(middleman.supplied_to_exporters, exporter.capacity - exporter.supplied_to_eu, 1000)
            exporter.supplied_to_eu += transfer
            middleman.supplied_to_exporters -= transfer

# Main simulation function
def monte_carlo_coffee_flows(comtrade_data, production_data, num_simulations=1000, verbose=False):
    results = []

    for _ in range(num_simulations):
        for country, export_data in comtrade_data.items():
            if country not in production_data:
                continue

            # Extract country data
            num_farmers = production_data[country]['num_farmers']
            est_production = production_data[country]['est_production']
            eu_exports = export_data['EU']
            total_exports = sum(export_data.values())

            # Generate entities
            farmers = generate_farmers(num_farmers, est_production)
            middlemen = generate_middlemen(max(10, num_farmers // 100), est_production)
            exporters = generate_exporters(max(5, len(middlemen) // 100), total_exports)

            # Assign farmers to middlemen
            assign_farmers_to_middlemen(farmers, middlemen)

            # Assign middlemen to exporters
            assign_middlemen_to_exporters(middlemen, exporters)

            # Calculate EU contribution
            eu_threshold = np.percentile([exp.capacity for exp in exporters], 80)  # Top 20% exporters
            eu_exporters = [exp for exp in exporters if exp.capacity >= eu_threshold]
            eu_supply = sum(exp.supplied_to_eu for exp in eu_exporters)

            # Estimate farmers selling to EU
            farmers_to_eu = sum(farmer.production for farmer in farmers) * (eu_supply / total_exports)
            farmers_selling_to_eu = farmers_to_eu / (est_production / num_farmers)

            # Collect results
            country_result = {
                "Country": country,
                "Total Farmers": num_farmers,
                "Total Production (kg)": est_production,
                "Exports to EU (kg)": eu_exports,
                "Estimated Exporters": len(exporters),
                "Estimated Middlemen": len(middlemen),
                "Estimated Farmers Selling to EU": round(farmers_selling_to_eu),
            }
            results.append(country_result)

            # Print results for the country if verbose is enabled
            if verbose:
                print(f"--- {country} ---")
                for key, value in country_result.items():
                    print(f"{key}: {value}")
                print("\n")

    return results

In [93]:
def monte_carlo_coffee_flows(comtrade_data, production_data, num_simulations=1000, verbose=False):
    results = []

    for _ in range(num_simulations):
        country_results = []

        for country, export_data in comtrade_data.items():
            if country not in production_data:
                continue

            # Extract country data
            num_farmers = production_data[country]['num_farmers']
            est_production = production_data[country]['est_production']
            eu_exports = export_data['EU']
            total_exports = sum(export_data.values())

            # Generate farmers' production (lognormal distribution)
            farmers_production = np.random.lognormal(mean=np.log(est_production / num_farmers),
                                                     sigma=0.5, size=min(5000, num_farmers))
            farmers_production = np.maximum(farmers_production, 10)  # Minimum 10 kg
            farmers_production = farmers_production / farmers_production.sum() * est_production

            # Generate middlemen's buying capacity (normal distribution)
            num_middlemen = max(10, num_farmers // 100)
            middlemen_capacity = np.random.normal(loc=0.01 * est_production, 
                                                  scale=0.005 * est_production, size=num_middlemen)
            middlemen_capacity = np.clip(middlemen_capacity, 1000, None)

            # Generate exporters' capacity (Pareto distribution)
            num_exporters = max(5, num_middlemen // 100)
            pareto_alpha = 1.5
            exporters_capacity = (np.random.pareto(pareto_alpha, num_exporters) + 1) * max(20000, 0.001 * total_exports)
            exporters_capacity = np.clip(exporters_capacity, 20000, None)

            # Assign farmers -> middlemen
            farmer_to_middleman = np.random.choice(num_middlemen, size=farmers_production.shape[0], replace=True)
            middleman_supply = np.zeros(num_middlemen)
            for i, farmer_output in enumerate(farmers_production):
                middleman_supply[farmer_to_middleman[i]] += farmer_output

            # Assign middlemen -> exporters
            middleman_to_exporter = np.random.choice(num_exporters, size=middleman_supply.shape[0], replace=True)
            exporter_supply = np.zeros(num_exporters)
            for i, middleman_output in enumerate(middleman_supply):
                exporter_supply[middleman_to_exporter[i]] += middleman_output

            # Calculate EU contribution
            eu_threshold = np.percentile(exporters_capacity, 80)  # Top 20% exporters
            eu_exporters = np.where(exporters_capacity >= eu_threshold)[0]
            eu_supply = exporter_supply[eu_exporters].sum()

            # Estimate number of farmers contributing to EU
            farmers_to_eu = farmers_production.sum() * (eu_supply / total_exports)
            farmers_selling_to_eu = farmers_to_eu / (est_production / num_farmers)

            # Collect results for the country
            country_result = {
                "Country": country,
                "Total Farmers": num_farmers,
                "Total Production (kg)": est_production,
                "Exports to EU (kg)": eu_exports,
                "Estimated Exporters": num_exporters,
                "Estimated Middlemen": num_middlemen,
                "Estimated Farmers Selling to EU": round(farmers_selling_to_eu),
            }
            results.append(country_result)

            # Print results for the country if verbose is enabled
            if verbose:
                print(f"--- {country} ---")
                for key, value in country_result.items():
                    print(f"{key}: {value}")
                print("\n")

        # Add country results to overall results
        #results.append(country_results)

    # Return results as a list of DataFrames
    return results

In [101]:
results = monte_carlo_coffee_flows(comtrade_data, production_data, num_simulations=1, verbose=True)

KeyboardInterrupt: 

In [96]:
pd.DataFrame(results)

,Country,Total Farmers,Total Production (kg),Exports to EU (kg),Estimated Exporters,Estimated Middlemen,Estimated Farmers Selling to EU
0,Bolivia,17682,6011880,495391.0,5,176,12258
1,Brazil,287300,3978000008,954149791.0,28,2873,117431
2,Burundi,583260,12900225,11347278.0,58,5832,101863
3,China,147480,118823606,28086863.0,14,1474,58332
4,Colombia,547195,695999991,151467276.0,54,5471,107820
5,Congo (Kinshasa),199800,16500003,4801510.0,19,1998,96980
6,Costa Rica,32899,86164791,13138902.0,5,328,7859
7,Cote d'Ivoire,149770,81000000,25111729.0,14,1497,57819
8,Ecuador,52082,22202858,394616.0,5,520,137298
9,El Salvador,20010,40200001,10616954.0,5,200,4107
